# Word Sense Disambiguation 

In [15]:
import pandas as pd
txt_file = pd.read_excel("Bank.xlsx")
txt_file=txt_file.drop([93,94,95,96,97,98,99,100],axis=0)
df=txt_file.drop(['Unnamed: 0'],axis=1)
df.columns

Index(['Sentences for Training ', 'Sense of the word "Bank"'], dtype='object')

In [18]:
corpus = df['Sentences for Training ']
sense = df['Sense of the word "Bank"']
sense

0     Financial Institution
1              River Border
2     Financial Institution
3              River Border
4     Financial Institution
              ...          
88             River Border
89             River Border
90             River Border
91    Financial Institution
92             River Border
Name: Sense of the word "Bank", Length: 93, dtype: object

In [19]:
import string
import nltk ,re
import math
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sankaranarayanan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sankaranarayanan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
stop_words = set(stopwords.words('english'))
for i in range(0, len(corpus)):
    corpus[i] = nltk.word_tokenize(corpus[i])
corpus

0        [I, deposited, my, paycheck, at, the, bank, .]
1     [The, bank, of, the, river, was, eroded, by, t...
2         [She, works, at, the, bank, as, a, teller, .]
3     [Let, 's, have, a, picnic, by, the, bank, of, ...
4       [The, bank, approved, my, loan, application, .]
                            ...                        
88    [The, children, built, sandcastles, along, the...
89    [The, otters, played, along, the, grassy, bank...
90    [The, bank, of, the, river, was, a, popular, s...
91    [I, need, to, deposit, this, check, into, my, ...
92    [The, children, skipped, stones, across, the, ...
Name: Sentences for Training , Length: 93, dtype: object

In [21]:
ws= []
filtered_corpus = []
for i in corpus:
    stop = []
    for j in i:
        if j not in stop_words and j != "'s":
            ws.append(j)
        else:
            stop.append(j)
    filtered_corpus.append(i)
    for k in stop:
        filtered_corpus[-1].remove(k)
ws

['I',
 'deposited',
 'paycheck',
 'bank',
 '.',
 'The',
 'bank',
 'river',
 'eroded',
 'heavy',
 'rains',
 '.',
 'She',
 'works',
 'bank',
 'teller',
 '.',
 'Let',
 'picnic',
 'bank',
 'river',
 '.',
 'The',
 'bank',
 'approved',
 'loan',
 'application',
 '.',
 'The',
 'fisherman',
 'cast',
 'line',
 'near',
 'bank',
 'river',
 '.',
 'We',
 'need',
 'transfer',
 'funds',
 'banks',
 '.',
 'The',
 'children',
 'built',
 'sandcastle',
 'bank',
 'river',
 '.',
 'The',
 'bank',
 'open',
 '9',
 '5',
 'pm',
 '.',
 'Ducks',
 'swimming',
 'peacefully',
 'along',
 'bank',
 'river',
 '.',
 'I',
 'need',
 'check',
 'account',
 'balance',
 'bank',
 '.',
 'The',
 'campsite',
 'situated',
 'bank',
 'river',
 '.',
 'She',
 'invested',
 'savings',
 'stock',
 'market',
 'bank',
 '.',
 'The',
 'trees',
 'shaded',
 'bank',
 'river',
 ',',
 'creating',
 'serene',
 'atmosphere',
 '.',
 'He',
 'withdrew',
 'cash',
 'ATM',
 'bank',
 '.',
 'The',
 'canoeists',
 'navigated',
 'along',
 'winding',
 'bank',
 'riv

In [42]:
vocab=list(set(ws))
len(vocab)

267

In [55]:
sense_data = {
    'Financial Institution':len(df[df['Sense of the word "Bank"']=='Financial Institution']),
    'River Border':len(df[df['Sense of the word "Bank"']=='River Border'])
}
sense_data

{'Financial Institution': 45, 'River Border': 48}

In [56]:
target_sentence = "We need to withdraw some cash from the bank for groceries."
target_token = nltk.word_tokenize(target_sentence)
target_ws = []
for i in target_token:
    if i not in stop_words:
        target_ws.append(i)
target_ws

['We', 'need', 'withdraw', 'cash', 'bank', 'groceries', '.']

In [59]:
class_count = {}
for i in target_ws:
    for k in sense_data.keys():
        class_count[i+" "+k] = 0
    for j in range(0, len(filtered_corpus)):
        word_count = count(filtered_corpus[j], i)
        class_count[i+" "+sense[j]]+=word_count
class_count

{'We Financial Institution': 3,
 'We River Border': 1,
 'need Financial Institution': 17,
 'need River Border': 0,
 'withdraw Financial Institution': 0,
 'withdraw River Border': 0,
 'cash Financial Institution': 1,
 'cash River Border': 0,
 'bank Financial Institution': 42,
 'bank River Border': 48,
 'groceries Financial Institution': 0,
 'groceries River Border': 0,
 '. Financial Institution': 45,
 '. River Border': 48}

In [69]:
result = {}
for i in sense_data.keys():
    prob = log(2,sense_data[i]/len(filtered_corpus))
    for j in target_ws:
        prob *= log(2,(class_count[j+" "+i]+1)/(sense_data[i]+len(vocab)))
    result[i] = prob

In [70]:
result

{'Financial Institution': 9.346713510542884e-06,
 'River Border': 4.199655819816815e-06}

In [71]:
max(result, key=result.get)

'Financial Institution'